In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import os
from collections import Counter

filename = "./Train.csv"
df = pd.read_csv(filename, usecols=['text', 'label'], dtype={'text': 'str', 'label': 'int64'})
dfX = df.loc[:, 'text']
dfY = df.loc[:, 'label']
dfX = dfX[:10000]
dfY = dfY[:10000]

vocabulary = Counter()
for title in dfX:
    words = title.split()
    validWords = filter(lambda x: len(x) <= 14, words)
    vocabulary.update(validWords)

# Truncate vocabulary
vocab_size = 1000
truncatedVocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

# Print out the 10 most common words and the number of times they occur
print("Most common words:", vocabulary.most_common()[:10])

# Convert words to tensor
words = tf.constant(truncatedVocabulary)

# Assign each word an ID
word_ids = tf.range(len(truncatedVocabulary), dtype=tf.int64)

# Create KeyValueTensor
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)

# Create lookup table
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

# Test the lookup table
#testArr = "China and Iraq are in the dataset".split()
#testRes = table.lookup(tf.constant(testArr))
#print("Test result:", testRes)

print("done");

2022-02-05 14:11:22.731210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-05 14:11:22.731305: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Most common words: [('the', 112780), ('a', 61522), ('and', 60362), ('of', 56663), ('to', 52397), ('is', 40621), ('in', 34059), ('I', 26135), ('that', 25365), ('this', 23018)]
done


2022-02-05 14:11:28.974738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-05 14:11:28.974873: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-05 14:11:28.974946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (penguin): /proc/driver/nvidia/version does not exist
2022-02-05 14:11:28.976073: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Convert sentences to arrays of word ids
data = []
for title in dfX:
    sample = tf.strings.split(title)
    processed = table.lookup(sample)
    data.append(processed.numpy())

# Create a ragged tensor and then convert it to a padded dense tensor
ragged = tf.ragged.constant(data)
ragged = ragged.to_tensor(default_value=0)

# Make dataset
features = tf.constant(ragged)
print(features.shape)
labels = tf.constant(dfY)
print(labels.shape)

train_set = tf.data.Dataset.from_tensor_slices((features, labels)).batch(32).prefetch(1)
#print(train_set)

#print(next(train_set.batch(32).as_numpy_iterator())[0][0])
print('done')


(10000, 2470)
(10000,)
done


2022-02-05 14:12:03.880993: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98800000 exceeds 10% of free system memory.


In [ ]:
# Create model
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None], mask_zero=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, batch_size=32, epochs=5)
print('done')

2022-02-05 14:12:05.826068: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98800000 exceeds 10% of free system memory.
2022-02-05 14:12:05.971608: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 98800000 exceeds 10% of free system memory.


Epoch 1/5


2022-02-05 14:12:12.903489: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 40468480 exceeds 10% of free system memory.
2022-02-05 14:12:13.097999: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 40468480 exceeds 10% of free system memory.


  5/313 [..............................] - ETA: 42:12 - loss: 0.6900 - accuracy: 0.5312

In [ ]:
testArr = tf.strings.split("This was pretty much my favorite movie ive ever seen")
test = table.lookup(testArr)
zero_padding = tf.zeros(tf.shape(features)[1] - tf.shape(test)[0], dtype=tf.int64)
a_padded = tf.concat([test, zero_padding],0)
a_padded = a_padded.numpy().reshape(1,-1);
#print("Prediction input: ", a_padded)
prediction = model.predict(a_padded)
print(prediction)
print("Positive:", prediction[0] > 0.6)
